In [1]:
import pandas as pd

from experiment.utils.tables.reports_raw_table import ReportsRawTable
from experiment.utils import dbutils
from experiment.utils import transformation

In [2]:
db = dbutils.DatabaseUtils()

In [3]:
# read the csv file
raw_ct_brain = transformation.get_project_root() / "tmp" / "data" / "raw_ct_brain.csv"
df = pd.read_csv(raw_ct_brain)
df.drop_duplicates(subset=['PROTOKOL_NO'], inplace=True)

In [4]:
reports_raw, Base = ReportsRawTable()
Base.metadata.create_all(db.engine)

data_to_insert = []
cols_to_upsert = ["patient_no", "full_name", "report_original"]
for _, row in df.iterrows():
    data_to_insert.append(
        {
            "report_id": row["PROTOKOL_NO"],
            "patient_no": row["HASTA_NO"],
            "full_name": row["AD_SOYAD"],
            "report_original": row["ACIKLAMA"],
        }
    )

db.upsert_values(reports_raw, data_to_insert, cols_to_upsert, ["report_id"])